In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
vocab_dir = "drive/My Drive/colab/cnews中文文本分类RNN/cnews.vocab.txt"
train_dir = "drive/My Drive/colab/cnews中文文本分类RNN/cnews.train.txt"
val_dir = "drive/My Drive/colab/cnews中文文本分类RNN/cnews.val.txt"
test_dir = "drive/My Drive/colab/cnews中文文本分类RNN/cnews.test.txt"

In [3]:
import torch
import numpy as np
import pandas as pd
from torch import optim,nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.utils.data import DataLoader
import keras

In [4]:
def read_vocab(vocab_dir):
    with open(vocab_dir,"r",encoding="utf-8",errors="ignore") as f:
        # print(f.read())
        words = [word.strip() for word in f.readlines()]
        # print(words)
    word_to_id = dict(zip(words,range(len(words))))
    # print(word_to_id)
    return words,word_to_id
words,word_to_id = read_vocab(vocab_dir)
print(words)
print(word_to_id)

['<PAD>', '，', '的', '。', '一', '是', '在', '0', '有', '不', '了', '中', '1', '人', '大', '、', '国', '', '2', '这', '上', '为', '个', '“', '”', '年', '学', '时', '我', '地', '和', '以', '到', '出', '来', '会', '行', '发', '：', '对', '们', '要', '生', '家', '他', '能', '也', '业', '金', '3', '成', '可', '分', '多', '现', '5', '就', '场', '新', '后', '于', '下', '日', '经', '市', '前', '过', '方', '得', '作', '月', '最', '开', '房', '》', '《', '高', '9', '8', '.', '而', '比', '公', '4', '说', ')', '将', '(', '都', '资', 'e', '6', '基', '用', '面', '产', '还', '自', '者', '本', '之', '美', '很', '同', '', '7', '部', '进', '但', '主', '外', '动', '机', '元', '理', '加', 'a', '全', '与', '实', '影', '好', '小', '间', '其', '天', '定', '表', '力', '如', '次', '合', '长', 'o', '体', '价', 'i', '所', '内', '子', '目', '电', '-', '当', '度', '品', '看', '期', '关', '更', 'n', '等', '工', '然', '斯', '重', '些', '球', '此', '里', '利', '相', '情', '投', '点', '没', '因', '已', '三', '心', '特', '明', '位', '两', '法', '从', '入', '名', '万', '手', '计', '性', '事', '只', '样', '示', 'r', '种', '报', '海', '平', '数', '%', '第', '并', '色', '建', '据', '提', '商

In [5]:
def read_category():
    categories = ['体育','财经','房产','家居','教育','科技','时尚','时政','游戏','娱乐']
    cat_to_id = dict(zip(categories,range(len(categories))))
    # print(cat_to_id)
    return categories,cat_to_id
categories,cat_to_id = read_category()
print(categories)
print(cat_to_id)

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
{'体育': 0, '财经': 1, '房产': 2, '家居': 3, '教育': 4, '科技': 5, '时尚': 6, '时政': 7, '游戏': 8, '娱乐': 9}


In [6]:
def process_file(filename,word_to_id,cat_to_id,max_length=600):
  contents,labels=[],[]
  with open(filename,"r",encoding="utf-8",errors="ignore") as f:
    for line in f:
      try:
        label,content = line.strip().split("\t")
        if content:
          contents.append(list(content))
          labels.append(label)
      except:
        pass
    data_id,label_id = [],[]
    for i in range(len(contents)):
      data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
      label_id.append(cat_to_id[labels[i]])
    #keras.preprocessing.sequence.pad_sequences将序列转化为经过填充以后的长度相同的新序列
    x_pad = keras.preprocessing.sequence.pad_sequences(data_id,max_length)
    #keras.utils.to_categorical将整型的类别标签转为onehot编码，num_classses指定类别总数
    y_pad = keras.utils.to_categorical(label_id,num_classes=len(cat_to_id))
    return x_pad,y_pad
x_train,y_train = process_file(train_dir,word_to_id,cat_to_id,600)
x_val,y_val = process_file(val_dir,word_to_id,cat_to_id,600)

In [7]:
print(x_train)
print(y_train.shape)

[[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]
(50000, 10)


In [8]:
cuda = torch.device("cuda")
x_train,y_train = torch.LongTensor(x_train),torch.Tensor(y_train)
x_val,y_val = torch.LongTensor(x_val),torch.Tensor(y_val)

train_dataset = Data.TensorDataset(x_train,y_train)
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=200,shuffle=True)
val_dataset = Data.TensorDataset(x_val,y_val)
val_loader = Data.DataLoader(dataset=val_dataset,batch_size=200,shuffle=True)

In [9]:
class TextRNN(nn.Module):
  def __init__(self):
    super(TextRNN,self).__init__()
    self.embedding = nn.Embedding(5000,64)
    self.rnn = nn.GRU(input_size=64,hidden_size=128,num_layers=2,bidirectional=True)
    self.f1 = nn.Sequential(nn.Linear(256,128),
                 nn.Dropout(0.8),
                 nn.ReLU())
    self.f2 = nn.Sequential(nn.Linear(128,10),
                 nn.Softmax())
  def forward(self,x):
    x = self.embedding(x)
    x,_ = self.rnn(x)
    x = F.dropout(x,p=0.8)
    x = self.f1(x[:,-1,:])
    return self.f2(x)

In [19]:
model = TextRNN()
model = model.cuda()
Loss = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
best_val_acc = 0
for epoch in range(50):
  for step,(x_batch,y_batch) in enumerate(train_loader):
    x,y = x_batch.cuda(),y_batch.cuda()
    out = model(x)
    loss = Loss(out,y)
    print(step,loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    accuracy = np.mean((torch.argmax(out,1)==torch.argmax(y,1)).cpu().numpy())
    print(epoch,accuracy)

  if (epoch+1)%5 == 0:
    for step,(x_batch,y_batch) in enumerate(val_loader):
      x,y = x_batch.cuda(),y_batch.cuda()
      out = model(x)
      accuracy = np.mean((torch.argmax(out,1)==torch.argmax(y,1)).cpu().numpy())
      if accuracy > best_val_acc:
        best_val_acc = accuracy
        torch.save(model.state_dict(),"drive/My Drive/colab/cnews中文文本分类RNN/model_params.pkl")

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


流式输出内容被截断，只能显示最后 5000 行内容。
40 0.39
1 0.708747386932373
40 0.445
2 0.7108868360519409
40 0.405
3 0.7117980718612671
40 0.39
4 0.7137266397476196
40 0.37
5 0.7125728726387024
40 0.405
6 0.712820827960968
40 0.395
7 0.7135151624679565
40 0.37
8 0.7130241990089417
40 0.38
9 0.7163063883781433
40 0.36
10 0.7134259343147278
40 0.385
11 0.7114326357841492
40 0.39
12 0.7097520232200623
40 0.42
13 0.7141027450561523
40 0.38
14 0.7164590358734131
40 0.37
15 0.7059969902038574
40 0.485
16 0.7127731442451477
40 0.38
17 0.7147420048713684
40 0.385
18 0.713313102722168
40 0.37
19 0.7167366147041321
40 0.36
20 0.7143322825431824
40 0.395
21 0.7141116857528687
40 0.375
22 0.7076884508132935
40 0.45
23 0.7076100707054138
40 0.45
24 0.713888943195343
40 0.38
25 0.7110905051231384
40 0.4
26 0.710749626159668
40 0.44
27 0.7101316452026367
40 0.43
28 0.7134405374526978
40 0.38
29 0.713144838809967
40 0.395
30 0.7099137902259827
40 0.435
31 0.7155152559280396
40 0.34
32 0.7126067876815796
40 0.385
33 0.7107

In [27]:
content = ['《时光重返四十二难》恶搞唐增取经一款时下最热门的动画人物：猪猪侠，加上创新的故事背景，震撼的操作快感，成就了这部恶搞新作，现正恶搞上市，玩家们抢先赶快体验快感吧。游戏简介：被时光隧道传送到208年的猪猪侠，必须经历六七四十二难的考验，才能借助柯伊诺尔大钻石的力量，开启时光隧道，重返2008年。在迷糊老师、菲菲公主的帮助下，猪猪侠接受了挑战，开始了这段充满了关心和情谊的旅程。    更多精彩震撼感觉，立即下载该款游戏尽情体验吧。玩家交流才是王道，讯易游戏玩家交流中心 QQ群：6306852-----------------生活要有激情，游戏要玩多彩(多彩游戏)。Colourfulgame (多彩游戏)，让你看看快乐游戏的颜色！精品推荐：1：《钟馗传》大战无头关羽，悲壮的剧情伴随各朝英灵反攻地府！2：《中华群英》将和赵云，项羽，岳飞等猛将作战，穿越各朝代抗击日寇。良品推荐：1：《赌王争霸之斗地主》易飞会在四角恋中会选择谁？是否最终成赌神呢？2：勇者后裔和魔王紧缠一起，前代恩怨《圣火伏魔录》将为您揭示一切。  3：颠覆传统概念，恶搞+非主流？！誓必弄死搞残为止《爆笑飞行棋》。4：《中国象棋残局大师》快棋和人机模式让畅快对弈！一切“多彩游戏”资讯，点击Colourfulgame官网http://www.colourfulgame.com一切“多彩游戏”感言，交流Colourfulgame论坛http://121.33.203.124/forum/【客服邮箱】：xunyiwangluo@126.com">xunyiwangluo@126.com">xunyiwangluo@126.com【客服热线】：020-87588437']
for i in content:
  data = [word_to_id[x] for x in content if x in cat_to_id]
  data = keras.preprocessing.sequence.pad_sequences([data],600)
  data = torch.LongTensor(data)
  data = data.cuda()
  y_pred = model(data)
  class_index = torch.argmax(y_pred[0]).item()
  print(i,":",categories[class_index])

《时光重返四十二难》恶搞唐增取经一款时下最热门的动画人物：猪猪侠，加上创新的故事背景，震撼的操作快感，成就了这部恶搞新作，现正恶搞上市，玩家们抢先赶快体验快感吧。游戏简介：被时光隧道传送到208年的猪猪侠，必须经历六七四十二难的考验，才能借助柯伊诺尔大钻石的力量，开启时光隧道，重返2008年。在迷糊老师、菲菲公主的帮助下，猪猪侠接受了挑战，开始了这段充满了关心和情谊的旅程。    更多精彩震撼感觉，立即下载该款游戏尽情体验吧。玩家交流才是王道，讯易游戏玩家交流中心 QQ群：6306852-----------------生活要有激情，游戏要玩多彩(多彩游戏)。Colourfulgame (多彩游戏)，让你看看快乐游戏的颜色！精品推荐：1：《钟馗传》大战无头关羽，悲壮的剧情伴随各朝英灵反攻地府！2：《中华群英》将和赵云，项羽，岳飞等猛将作战，穿越各朝代抗击日寇。良品推荐：1：《赌王争霸之斗地主》易飞会在四角恋中会选择谁？是否最终成赌神呢？2：勇者后裔和魔王紧缠一起，前代恩怨《圣火伏魔录》将为您揭示一切。  3：颠覆传统概念，恶搞+非主流？！誓必弄死搞残为止《爆笑飞行棋》。4：《中国象棋残局大师》快棋和人机模式让畅快对弈！一切“多彩游戏”资讯，点击Colourfulgame官网http://www.colourfulgame.com一切“多彩游戏”感言，交流Colourfulgame论坛http://121.33.203.124/forum/【客服邮箱】：xunyiwangluo@126.com">xunyiwangluo@126.com">xunyiwangluo@126.com【客服热线】：020-87588437 : 游戏


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
